In [3]:
import itertools
import numpy as np
import time

# ================== ЗАДАНИЕ 1 ==================
print("1. Генерация GL(2, Z₂)")
all_matrices = list(itertools.product([0,1], repeat=4))
gl2_z2 = [np.array([[a,b],[c,d]]) for a,b,c,d in all_matrices if (a*d - b*c) % 2 != 0]
print(f"Порядок группы: {len(gl2_z2)}")
for i, M in enumerate(gl2_z2, 1):
    print(f"M{i}:\n{M}")

# ================== ЗАДАНИЕ 2 ==================
print("\n2. Проверка групповых свойств")
I = np.eye(2, dtype=int)
closed = all(any(np.array_equal(np.mod(np.dot(A,B),2), M) for M in gl2_z2) for A in gl2_z2 for B in gl2_z2)
assoc = np.array_equal(np.mod(np.dot(np.dot(gl2_z2[0],gl2_z2[1]),gl2_z2[2]),2), 
                       np.mod(np.dot(gl2_z2[0],np.dot(gl2_z2[1],gl2_z2[2])),2))
identity = any(np.array_equal(M, I) for M in gl2_z2)
inverses = all(np.array_equal(np.mod(np.dot(M, np.array([[M[1,1], -M[0,1]],[-M[1,0], M[0,0]]]) % 2),2), I) for M in gl2_z2)
print(f"Замкнутость: {closed}, Ассоциативность: {assoc}, Нейтральный: {identity}, Обратные: {inverses}")

# ================== ЗАДАНИЕ 3 ==================
print("\n3. Таблица Кэли")
print("   " + " ".join(f"M{j+1}" for j in range(len(gl2_z2))))
for i, A in enumerate(gl2_z2):
    row = f"M{i+1}|" + "".join(f" M{np.where([np.array_equal(np.mod(np.dot(A,B),2), M) for M in gl2_z2])[0][0]+1}" for B in gl2_z2)
    print(row)

# ================== ЗАДАНИЕ 4 ==================
print("\n4. Сравнение производительности")
print("Метод 1: Наивный перебор (прямой расчет определителя для каждой матрицы)")
print("Метод 2: Наша оптимизация (использование формулы для Z₂)")

def naive_method(n):
    """Наивный перебор: вычисление определителя для каждой матрицы"""
    start = time.time()
    mats = list(itertools.product([0,1], repeat=n*n))
    count = 0
    for mat in mats:
        matrix = np.array(mat).reshape(n, n)
        if np.linalg.det(matrix) % 2 != 0:
            count += 1
    return time.time() - start, count

def optimized_method(n):
    """Наш метод: использование свойств Z₂"""
    start = time.time()
    mats = list(itertools.product([0,1], repeat=n*n))
    if n == 2:
        # Для 2x2 используем простую формулу
        count = sum(1 for a,b,c,d in mats if (a*d - b*c) % 2 != 0)
    else:
        # Для больших матриц используем тот же наивный метод
        count = sum(1 for mat in mats if np.linalg.det(np.array(mat).reshape(n,n)) % 2 != 0)
    return time.time() - start, count

# Сравниваем для n=2,3,4
print("\nСравнение времени выполнения (секунды):")
print("n | Наивный метод | Наш метод | Ускорение")
print("-" * 40)

for n in [2, 3, 4]:
    time_naive, count_naive = naive_method(n)
    time_opt, count_opt = optimized_method(n)
    speedup = time_naive / time_opt if time_opt > 0 else float('inf')
    
    print(f"{n} | {time_naive:.6f} | {time_opt:.6f} | {speedup:.1f}x")
    
    # Проверяем, что результаты совпадают
    assert count_naive == count_opt, f"Результаты не совпадают для n={n}"

print("\nВывод: Для n=2 наш метод значительно быстрее, так как")
print("использует простую формулу вместо вычисления определителя")
print("Для n>2 оба метода одинаковы (используют np.linalg.det)")

1. (наивный перебор) Генерация GL(2, Z₂)
Порядок группы: 6
M1:
[[0 1]
 [1 0]]
M2:
[[0 1]
 [1 1]]
M3:
[[1 0]
 [0 1]]
M4:
[[1 0]
 [1 1]]
M5:
[[1 1]
 [0 1]]
M6:
[[1 1]
 [1 0]]

2. Проверка групповых свойств
Замкнутость: True, Ассоциативность: True, Нейтральный: True, Обратные: True

3. Таблица Кэли
   M1 M2 M3 M4 M5 M6
M1| M3 M5 M1 M6 M2 M4
M2| M4 M6 M2 M5 M1 M3
M3| M1 M2 M3 M4 M5 M6
M4| M2 M1 M4 M3 M6 M5
M5| M6 M4 M5 M2 M3 M1
M6| M5 M3 M6 M1 M4 M2

4. Сравнение производительности
Метод 1: Наивный перебор (прямой расчет определителя для каждой матрицы)
Метод 2: Наша оптимизация (использование формулы для Z₂)

Сравнение времени выполнения (секунды):
n | Наивный метод | Наш метод | Ускорение
----------------------------------------
2 | 0.001010 | 0.000000 | infx
3 | 0.002979 | 0.003990 | 0.7x
4 | 0.449173 | 0.424207 | 1.1x

Вывод: Для n=2 наш метод значительно быстрее, так как
использует простую формулу вместо вычисления определителя
Для n>2 оба метода одинаковы (используют np.linalg.det)
